In [1]:
%pip install vgamepad

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ----------------------------------- ---- 1.0/1.2 MB 12.5 MB/s eta 0:00:01
     ----------------------------------- ---- 1.0/1.2 MB 12.5 MB/s eta 0:00:01
     ----------------------------------- ---- 1.0/1.2 MB 12.5 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 1.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for vgamepad: filename=vgamepad-0.1.0-py3-none-any.whl size=1169219 sha256=b87f4690a99baa8916094d53b7edb1c77a76b02f14136b1492b40170af1bf840
  Stored in directory: c:\users\wave\appdata\local\pip\cache\wheels\56\5d\a8\db3cf10072d8b2d37377dca25f80caedae9e6df5d6d70966f4


In [2]:
import hid

for device in hid.enumerate():
    print(f"0x{device['vendor_id']:04x}:0x{device['product_id']:04x} {device['product_string']}")

0x04f3:0x3147 HIDI2C Device
0x8087:0x0a1e 
0x045e:0x02ff Controller (Xbox One For Windows)
0x04f3:0x3147 HIDI2C Device
0x04f3:0x3147 HIDI2C Device
0x045e:0x0000 
0x045e:0x0000 
0x045e:0x0000 
0x045e:0x0b02 Controller (Xbox One For Windows)
0x04f3:0x3147 HIDI2C Device


In [5]:
gamepad = hid.device()
gamepad.open(0x0079, 0x0011) # The vendor and product IDs of the gamepad
gamepad.set_nonblocking(True)

print(gamepad)

In [6]:
import vgamepad as vg

virtualGamepad = vg.VX360Gamepad()

In [8]:
BIT_NAMES = {}

# Load the bit names from a file
with open("bit_names.txt", "r") as f:
    for line in f:
        bit, name = line.strip().split(" ", 1)
        BIT_NAMES[int(bit)] = name

print(f"Loaded {len(BIT_NAMES)} bit names: {BIT_NAMES}")

Loaded 11 bit names: {46: 'Up arrow', 47: 'Right arrow', 44: 'Left arrow', 45: 'Down arrow', 53: 'Start', 52: 'Select', 39: 'Center', 51: 'Top right circle', 50: 'Top left X', 49: 'Bottom left triangle', 48: 'Bottom right square'}


In [18]:
from time import sleep
from math import sqrt

# BUTTON_TO_NT_INPUT = {
#     # Manually authored for now
#     "Up arrow": "LEFT_Y_NEG",
#     "Right arrow": "LEFT_X_POS",
#     "Left arrow": "LEFT_X_NEG",
#     "Down arrow": "LEFT_Y_POS",
#     "Start": "RIGHT_X_POS",
#     "Select": "RIGHT_X_NEG",
#     "Center": None, # N/A
#     "Top right circle": ["LEFT_Y_NEG", "LEFT_X_POS"],
#     "Top left X": ["LEFT_Y_NEG", "LEFT_X_NEG"],
#     "Bottom left triangle": ["LEFT_Y_POS", "LEFT_X_NEG"],
#     "Bottom right square": ["LEFT_Y_POS", "LEFT_X_POS"]
# }

def is_pressed(name):
    bit = 0
    for bit_bit, bit_name in BIT_NAMES.items():
        if bit_name == name:
            bit = bit_bit
            break
    pressed = report[bit // 8] & (1 << (bit % 8))
    return pressed != 0

while True:
    report = gamepad.read(64)
    if report:
        horizontal = 0
        vertical = 0
        turn = 0
        if is_pressed("Up arrow"):
            vertical += 1
        if is_pressed("Down arrow"):
            vertical -= 1
        if is_pressed("Left arrow"):
            horizontal -= 1
        if is_pressed("Right arrow"):
            horizontal += 1
        
        if is_pressed("Top right circle"):
            vertical += 1
            horizontal += 1
        if is_pressed("Top left X"):
            vertical += 1
            horizontal -= 1
        if is_pressed("Bottom left triangle"):
            vertical -= 1
            horizontal -= 1
        if is_pressed("Bottom right square"):
            vertical -= 1
            horizontal += 1
        
        if is_pressed("Start"):
            turn += 0.5
        if is_pressed("Select"):
            turn -= 0.5
        
        drive_magnitude = 0.6
        current_drive_mag = sqrt(horizontal * horizontal + vertical * vertical)
        if current_drive_mag == 0:
            current_drive_mag = 1
        horizontal /= current_drive_mag / drive_magnitude
        vertical /= current_drive_mag / drive_magnitude

        # if vertical != 0 or horizontal != 0:
        #     print(horizontal, vertical)

        virtualGamepad.left_joystick_float(horizontal, vertical)
        virtualGamepad.right_joystick_float(turn, 0)
        virtualGamepad.update()

KeyboardInterrupt: 

In [7]:
from time import sleep

virtualGamepad.press_button(button=vg.XUSB_BUTTON.XUSB_GAMEPAD_A)
virtualGamepad.press_button(button=vg.XUSB_BUTTON.XUSB_GAMEPAD_DPAD_LEFT)

virtualGamepad.update()

sleep(0.5)

virtualGamepad.release_button(button=vg.XUSB_BUTTON.XUSB_GAMEPAD_A)

virtualGamepad.update()

sleep(0.5)

virtualGamepad.left_trigger(value=100)  # value between 0 and 255
virtualGamepad.right_trigger(value=255)  # value between 0 and 255
virtualGamepad.left_joystick(x_value=-10000, y_value=0)  # values between -32768 and 32767
virtualGamepad.right_joystick(x_value=-32768, y_value=15000)  # values between -32768 and 32767

virtualGamepad.update()

sleep(0.5)

virtualGamepad.reset()
virtualGamepad.update()

In [ ]:
v